## **1. Importation des bibliothèques**

In [21]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, UpSampling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import cv2
import tqdm
from sklearn.metrics import accuracy_score, jaccard_score

In [22]:
import tensorflow as tf

print("Version TensorFlow :", tf.__version__)
print("GPU disponible :", tf.config.list_physical_devices('GPU'))


Version TensorFlow : 2.10.0
GPU disponible : [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## **2. Préparation des données avec Générateurs de Batches**
 - Dans cette cellule, nous chargeons les images RGB d'origine ainsi que les masques de segmentation à partir des dossiers train, val, et test qui sont déjà préparés. Les images sont redimensionnées à une taille plus petite (256x256) pour rendre l'entraînement plus efficace, puis normalisées.
  - Le jeu de données est composé de 2 dossiers:
● leftImg8bit : ce dossier contient les images RGB d’origines. Ce sont les données d’éntrée
● gtFine : ce dossier contient les masques de segmentation. Ce sont les données de sortie
- En ce qui concerne les données de sortie, il y a plusieurs types de fichiers, mais nous ne nous intéresserons qu’aux
fichiers dont le nom se termine par “_labelIds.png”. Ce sont les masques correspondant aux images
RGB.

In [28]:
import os
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing import image

# Définitions des chemins de fichiers avec chemins relatifs
base_dir = os.path.join(os.getcwd(), 'data')
image_dir_train = os.path.join(base_dir, 'leftImg8bit/train')
mask_dir_train = os.path.join(base_dir, 'gtFine/train')
image_dir_val = os.path.join(base_dir, 'leftImg8bit/val')
mask_dir_val = os.path.join(base_dir, 'gtFine/val')
image_dir_test = os.path.join(base_dir, 'leftImg8bit/test')
mask_dir_test = os.path.join(base_dir, 'gtFine/test')

# Générateur de données pour charger les images par batch
# Générateur de données pour charger les images par batch
class DataGenerator(Sequence):
    def __init__(self, image_dir, mask_dir, batch_size=16, img_size=(256, 256)):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.batch_size = batch_size
        self.img_size = img_size
        self.image_list = []

        # Parcours récursif des dossiers pour les images
        for root, _, files in os.walk(image_dir):
            for file in files:
                if file.endswith(('.jpg', '.jpeg', '.png')):
                    self.image_list.append(os.path.join(root, file))

        # Parcours récursif des dossiers pour les masques (uniquement `_labelIds.png`)
        self.mask_list = []
        for root, _, files in os.walk(mask_dir):
            for file in files:
                if file.endswith('_labelIds.png'):
                    self.mask_list.append(os.path.join(root, file))

        # S'assurer que les listes sont triées pour que chaque image corresponde au bon masque
        self.image_list.sort()
        self.mask_list.sort()

        self.indexes = np.arange(len(self.image_list))

    def __len__(self):
        return len(self.image_list) // self.batch_size

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        images, masks = [], []

        for i in batch_indexes:
            img_path = self.image_list[i]
            mask_path = self.mask_list[i]
            img = image.load_img(img_path, target_size=self.img_size)
            img = image.img_to_array(img) / 255.0
            mask = image.load_img(mask_path, color_mode="grayscale", target_size=self.img_size)
            mask = image.img_to_array(mask) / 255.0

            images.append(img)
            masks.append(mask)

        return np.array(images), np.array(masks)


# Création des générateurs de données pour train, validation et test
batch_size = 16
train_gen = DataGenerator(image_dir_train, mask_dir_train, batch_size=batch_size)
val_gen = DataGenerator(image_dir_val, mask_dir_val, batch_size=batch_size)
test_gen = DataGenerator(image_dir_test, mask_dir_test, batch_size=batch_size)


In [29]:
print(f"Nombre d'images dans le dossier d'entraînement : {len(train_gen.image_list)}")
print(f"Nombre de masques dans le dossier d'entraînement : {len(train_gen.mask_list)}")


Nombre d'images dans le dossier d'entraînement : 2975
Nombre de masques dans le dossier d'entraînement : 2975


In [30]:
for images, masks in train_gen:
    print(images.shape, masks.shape)
    break


(16, 256, 256, 3) (16, 256, 256, 1)


## **3. Développement et entraînement du modèle U-Net:**

**Explication :** Cette cellule contient la définition et la construction du modèle U-Net. Le modèle est entraîné sur les données divisées avec 10 epochs. U-Net est bien adapté à la segmentation d'images. Des callbacks ont été ajoutés pour effectuer un early stopping et sauvegarder le meilleur modèle.


In [17]:
# Fonction pour construire le modèle U-Net
def unet_model(input_size=(256, 256, 3)):
    inputs = Input(input_size)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    p2 = MaxPooling2D((2, 2))(c2)

    # Partie à la descente
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)

    # Partie remontante
    u4 = UpSampling2D((2, 2))(c3)
    u4 = concatenate([u4, c2])
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(u4)

    u5 = UpSampling2D((2, 2))(c4)
    u5 = concatenate([u5, c1])
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(u5)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c5)
    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model

# Compilation du modèle
model = unet_model()
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping et sauvegarde du meilleur modèle
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(os.path.join('models', 'unet_best_model.h5'), save_best_only=True, monitor='val_loss')

# Entraînement avec le générateur de données
def train_unet_on_gpu():
    with tf.device('/GPU:0'):  # Spécifie l'utilisation du GPU
        model.fit(train_gen, validation_data=val_gen, epochs=10, callbacks=[early_stopping, model_checkpoint])


## **4.  Comparaison des Modèles U-Net, VGG16-UNET, et U-Net Mini**

**Explication :** Ici, nous avons ajouté un modèle U-Net Mini, qui est une version plus légère et simplifiée du U-Net. Le modèle VGG16-UNET a été mis à jour pour inclure une couche de data augmentation interne afin de mieux généraliser les performances.


In [19]:
# Importation des bibliothèques nécessaires
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom

# Développement du modèle VGG16-UNET
def vgg16_unet_model(input_size=(256, 256, 3)):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_size)
    
    # Data augmentation intégrée
    inputs = Input(input_size)
    x = RandomFlip("horizontal")(inputs)
    x = RandomRotation(0.2)(x)
    x = RandomZoom(0.2)(x)
    
    # Passage de l'entrée dans le modèle VGG16
    block1_conv2 = base_model.get_layer('block1_conv2').output
    block2_conv2 = base_model.get_layer('block2_conv2').output
    block3_conv3 = base_model.get_layer('block3_conv3').output
    vgg_outputs = [block1_conv2, block2_conv2, block3_conv3]
    
    # Définir un nouveau modèle avec les sorties intermédiaires désirées
    vgg16_encoder = Model(inputs=base_model.input, outputs=vgg_outputs)
    
    # Passer les images par l'encodeur
    c1, c2, c3 = vgg16_encoder(x)
    
    # Partie remontante du U-Net avec des couches de décodage
    u4 = UpSampling2D((2, 2))(c3)
    u4 = concatenate([u4, c2])
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(u4)
    
    u5 = UpSampling2D((2, 2))(c4)
    u5 = concatenate([u5, c1])
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(u5)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c5)
    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model

vgg16_model = vgg16_unet_model()
vgg16_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Entraînement du modèle VGG16-UNET avec générateur de données
vgg16_checkpoint = ModelCheckpoint(os.path.join('models', 'vgg16_best_model.h5'), save_best_only=True, monitor='val_loss')

def train_vgg16_on_gpu():
    with tf.device('/GPU:0'):  # Spécifie l'utilisation du GPU
        vgg16_model.fit(train_gen, validation_data=val_gen, epochs=10, callbacks=[early_stopping, vgg16_checkpoint])



In [ ]:
# Développement du modèle U-Net Mini
def unet_mini_model(input_size=(256, 256, 3)):
    inputs = Input(input_size)
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    p2 = MaxPooling2D((2, 2))(c2)

    # Partie à la descente
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(p2)

    # Partie remontante
    u4 = UpSampling2D((2, 2))(c3)
    u4 = concatenate([u4, c2])
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(u4)

    u5 = UpSampling2D((2, 2))(c4)
    u5 = concatenate([u5, c1])
    c5 = Conv2D(32, (3, 3), activation='relu', padding='same')(u5)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c5)
    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model

unet_mini_model = unet_mini_model()
unet_mini_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Entraînement du modèle U-Net Mini
def train_unet_mini_on_gpu():
    with tf.device('/GPU:0'):
        unet_mini_model.fit(train_gen, validation_data=val_gen, epochs=10, callbacks=[early_stopping, unet_mini_checkpoint])

## 5. Conclusion, Comparaison et Visualisation des Modèles

** Cette cellule compare les trois modèles entraînés (U-Net classique, VGG16-UNET, et U-Net Mini) sur les métriques telles que l'accuracy, l'IoU, et le Dice Coefficient. Des graphiques sont créés pour visualiser les performances de chaque modèle. 

In [ ]:
# Comparaison des performances des trois modèles
# On retient le modèle avec la meilleure précision sur l'ensemble de validation

def compare_models(models, val_gen, test_gen):
    metrics = []
    for model in models:
        model_name = model.name
        loss, accuracy = model.evaluate(val_gen, verbose=0)
        y_pred = model.predict(test_gen)
        y_pred_binary = (y_pred > 0.5).astype(np.uint8)
        y_test = np.concatenate([y for _, y in test_gen], axis=0)
        
        iou = jaccard_score(y_test.flatten(), y_pred_binary.flatten(), average='macro')
        dice = (2 * np.sum(y_test.flatten() * y_pred_binary.flatten())) / (np.sum(y_test.flatten()) + np.sum(y_pred_binary.flatten()))
        
        metrics.append({
            'model': model_name,
            'loss': loss,
            'accuracy': accuracy,
            'iou': iou,
            'dice': dice
        })
    return metrics

# Comparer les modèles
models = [model, vgg16_model, unet_mini_model]
metrics = compare_models(models, val_gen, test_gen)

In [ ]:
# Visualisation des résultats
import pandas as pd
metrics_df = pd.DataFrame(metrics)

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(metrics_df['model'], metrics_df['accuracy'], label='Accuracy', marker='o')
plt.plot(metrics_df['model'], metrics_df['iou'], label='IoU', marker='o')
plt.plot(metrics_df['model'], metrics_df['dice'], label='Dice Coefficient', marker='o')
plt.title('Comparaison des métriques entre modèles')
plt.xlabel('Modèles')
plt.ylabel('Score')
plt.legend()

plt.subplot(1, 2, 2)
plt.bar(metrics_df['model'], metrics_df['loss'], color='red')
plt.title('Loss des différents modèles')
plt.xlabel('Modèles')
plt.ylabel('Loss')
plt.show()

## Visualisation des exemples de résultats pour chaque modèle

In [ ]:
# Fonction de visualisation des prédictions

def visualize_predictions(models, test_gen, num_examples=3):
    for model in models:
        print(f"Visualisation des prédictions pour le modèle: {model.name}")
        predictions = model.predict(test_gen)
        X_test, y_test = next(iter(test_gen))
        for i in range(num_examples):
            plt.figure(figsize=(12, 4))
            plt.subplot(1, 3, 1)
            plt.imshow(X_test[i])
            plt.title("Image d'origine")
            
            plt.subplot(1, 3, 2)
            plt.imshow(y_test[i].squeeze(), cmap='gray')
            plt.title("Masque réel")
            
            plt.subplot(1, 3, 3)
            plt.imshow(predictions[i].squeeze(), cmap='gray')
            plt.title("Masque prédit")
            
            plt.show()

# Visualiser quelques exemples de segmentation
visualize_predictions(models, test_gen)